## Algoritmo para clasificación de Palma - CDCol IDEAM

El flujo metodológico para la elaboración del mapa nacional de cobertura de palma de aceite 2017 se encuentra desarrollado sobre un algoritmo semi-automatizado de aprendizaje maquina (Random Forest) diseñado haciendo uso de la arquitectura del Cubo de datos para Colombia (CDCol) del IDEAM y escrito en lenguaje Python 2.7.

El conjunto de datos para entrenamiento corresponde a la elaboración de compuestos temporales de mediana (CTM) anuales para el año 2017, usando cinco bandas espectrales (Green, Reed, NIR, SWIR1 y SWIR2), elaborados de dos unidades de almacenamiento (Landsat 7 y Landsat 8). Sobre los CTM obtenidos se calcularon tres (3) índices temáticos que aportaron una mejor respuesta a la clasificación de la cobertura. Dentro de los índices seleccionados se encuentran el NDVI (Normalized Difference Vegetation Index) que permite separar la vegetación de los cultivos del brillo que produce el suelo, el GNDVI (Green NDVI) un índice variante del NDVI y permite establecer la diferencia la vegetación y el estado del suelo y el RVI (Ratio Vegetation Index) el cual permite determinar la diferencia entre la vegetación y as características del suelo. Se optó por la inclusión en el compuesto del DEM para entrenar el algoritmo ya que de acuerdo con el ambiente de siembra de los cultivos de Palma de Aceite las condiciones óptimas para el cultivo se requiere una altitud máxima de 500 msnm con pendientes menores a 23%, los suelos deben ser planos o con ondulamiento suave, suelos francos y con buen drenaje  (Mingorance, Minelli, & Du, 2004), por lo que esta variable brindaba aporte en el entrenamiento. La clasificación se realizó en las zonas libres del enmascaramiento de los pixeles correspondientes a cobertura boscosa, como mascara se usó el mapa oficial de Bosque / No Bosque 2017 elaborado por el IDEAM.

Las clases finales del mapa corrsponden a un mapa tematico de Palma/ No palma. El mapa se presenta en escala 1: 100.000, por lo que el mosaico final esta generalizado a la unidad minima de mapeo (UMM) de 1 ha. 

Algoritmo desarrollado en python 2.7, en la infraestructura desacoplado del Cubo de Datos de Imágenes de Satélite de Colombia.

## Atributos generados

In [1]:
execID=975
algorithm = "RandomForestCm"
version= "1.0"

## Parámetros comunes

In [2]:
products = ['LS8_OLI_LASRC','LS7_ETM_LEDAPS' ] #Productos sobre los que se hará la consulta (unidades de almacenamiento)
bands=["green","red","nir", "swir1","swir2"] #arreglo de bandas #"blue","green",
time_ranges = [("2017-01-01", "2017-12-31")] #Una lista de tuplas, cada tupla representa un periodo
#área sobre la cual se hará la consulta:
min_long = -75
min_lat = 9
max_long = -74
max_lat = 10

## Parámetros específicos del algoritmo

In [3]:
train_data_path = '/home/cubo/notebooks/975'
validation_data_path= '/home/cubo/notebooks/975'
normalized=True
minValid=1;

# Librerías

In [4]:
import datacube
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
import os
import re
import xarray as xr
import numpy as np
import gdal
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score

In [5]:
def enmascarar_entrenamiento(vector_data_path, cols, rows, geo_transform, projection, target_value=1):
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds

In [6]:
def rasterizar_entrenamiento(file_paths, rows, cols, geo_transform, projection):
    labeled_pixels = np.zeros((rows, cols))
    for i, path in enumerate(file_paths):
        label = i+1
        ds = enmascarar_entrenamiento(path, cols, rows, geo_transform, projection, target_value=label)
        band = ds.GetRasterBand(1)
        labeled_pixels += band.ReadAsArray()
        ds = None
    return labeled_pixels

In [7]:
def exportar(fname, data, geo_transform, projection):
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None

# Consulta

Consulta sobre las diferentes unidades y aplica la máscara de nubes adecuada. 

In [8]:
nodata=-9999
#Definir las funciones necesatias para el algoritmo
def isin(element, test_elements, assume_unique=False, invert=False):
    "definiendo la función isin de numpy para la versión anterior a la 1.13, en la que no existe"
    element = np.asarray(element)
    return np.in1d(element, test_elements, assume_unique=assume_unique,
                invert=invert).reshape(element.shape)

### Máscara de nubes

Con el nuevo formato, los valores de `pixel_qa` dependen del producto. Para crear la máscara de nubes, se determinan los valores válidos para el producto actual y se usa la banda `pixel_qa` para generar un arreglo de datos booleanos: Para cada posición, si el valor de pixel_qa está en la lista de valores válidos será `True`, en caso contrario será `False`.

In [9]:
kwargs={}
dc = datacube.Datacube(app="{}_{}_{}".format(algorithm,version,execID))
for product in products:
    i=0
    validValues=set()
    if product=="LS7_ETM_LEDAPS":
        validValues=[66,68,130,132]
    elif product == "LS8_OLI_LASRC":
        validValues=[322, 386, 834, 898, 1346, 324, 388, 836, 900, 1348]
    for tr in time_ranges:
        _data = dc.load(product=product, longitude=(min_long, max_long), latitude=(min_lat, max_lat), time=tr)
        if len(_data.data_vars)==0:
            break
        cloud_mask=isin(_data["pixel_qa"].values, validValues)
        for band in bands:
            _data[band].values=np.where(np.logical_and(_data.data_vars[band]!=nodata,cloud_mask),_data.data_vars[band], np.nan)
        _undesired=list(set(_data.keys())-set(bands+['latitude','longitude','time']))
        _data=_data.drop(_undesired)
            
        if "xarr"+str(i) in kwargs:
            kwargs["xarr"+str(i)]=xr.concat([kwargs["xarr"+str(i)],_data.copy(deep=True)], 'time')
        else:
            kwargs["xarr"+str(i)]=_data
    i+=1
del _data

/home/cubo/anaconda2/lib/python2.7/_abcoll.py:410: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  return list(self)
/home/cubo/anaconda2/lib/python2.7/_abcoll.py:410: FutureWarning: calling len() on an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Call len() on the Dataset.variables property instead, like ``len(ds.variables)``, to preserve existing behavior in a forwards compatible manner.
  return list(self)


In [10]:
_undesired

[u'blue',
 u'solar_azimuth',
 u'sensor_zenith',
 u'radsat_qa',
 u'atmos_opacity',
 u'solar_zenith',
 u'pixel_qa',
 u'cloud_qa',
 u'sensor_azimuth']

In [11]:
#El algoritmo recibe los productos como xarrays en variablles llamadas xarr0, xarr1, xarr2... 
xarr0=kwargs["xarr0"]
del kwargs

In [12]:
xarr0

<xarray.Dataset>
Dimensions:    (latitude: 3687, longitude: 3704, time: 147)
Coordinates:
  * latitude   (latitude) float64 10.0 10.0 10.0 9.999 9.999 9.999 9.998 ...
  * longitude  (longitude) float64 -75.0 -75.0 -75.0 -75.0 -75.0 -75.0 -75.0 ...
  * time       (time) datetime64[ns] 2017-01-09T15:11:20 2017-01-09T15:11:44 ...
Data variables:
    green      (time, latitude, longitude) float64 nan nan nan nan nan nan ...
    red        (time, latitude, longitude) float64 nan nan nan nan nan nan ...
    nir        (time, latitude, longitude) float64 nan nan nan nan nan nan ...
    swir1      (time, latitude, longitude) float64 nan nan nan nan nan nan ...
    swir2      (time, latitude, longitude) float64 nan nan nan nan nan nan ...
Attributes:
    crs:      EPSG:4326

# Procesamiento

El algoritmo debe ser auto contenido. Puede importar y usar las librerías disponibles en CDCol, por ejemplo: 

- scikit-learn
- numpy
- xarray
- pycurl
- pandas
- nltk

In [13]:
dc = datacube.Datacube(app="{}_{}_{}".format(algorithm,version,execID))

_data = dc.load(product="FNF_COL_UTM", longitude=(min_long, max_long), latitude=(min_lat, max_lat), time=("2017-01-01", "2017-12-31"))

In [14]:
_data

<xarray.Dataset>
Dimensions:    (latitude: 3687, longitude: 3704, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 2017-01-01
  * latitude   (latitude) float64 10.0 10.0 10.0 9.999 9.999 9.999 9.998 ...
  * longitude  (longitude) float64 -75.0 -75.0 -75.0 -75.0 -75.0 -75.0 -75.0 ...
Data variables:
    fnf_mask   (time, latitude, longitude) int8 2 2 2 2 2 2 2 2 2 2 2 2 2 2 ...
Attributes:
    crs:      EPSG:4686

In [15]:
_data2 = dc.load(product="DEM_Mosaico", longitude=(min_long, max_long), latitude=(min_lat, max_lat), time=("2013-01-01", "2013-12-31"))

In [16]:
dem=_data2["dem"][0].values
del _data2

## Compuesto temporal de medianas

In [17]:
medians={} 
for band in bands:
    datos=xarr0[band].values
    allNan=~np.isnan(datos) #Una mascara que indica qué datos son o no nan. 
    if normalized: #Normalizar, si es necesario.
        #Para cada momento en el tiempo obtener el promedio y la desviación estándar de los valores de reflectancia
        m=np.nanmean(datos.reshape((datos.shape[0],-1)), axis=1)
        st=np.nanstd(datos.reshape((datos.shape[0],-1)), axis=1)
        # usar ((x-x̄)/st) para llevar la distribución a media 0 y desviación estándar 1, 
        # y luego hacer un cambio de espacio para la nueva desviación y media. 
        datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)
    #Calcular la mediana en la dimensión de tiempo 
    medians[band]=np.nanmedian(datos,0) 
    #Eliminar los valores que no cumplen con el número mínimo de pixeles válidos dado. 
    medians[band][np.sum(allNan,0)<minValid]=np.nan
    medians[band][_data["fnf_mask"].values[0]==1]=np.nan
del datos
del _data

/home/cubo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: Mean of empty slice
/home/cubo/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [18]:
medians["dem"]=dem

In [19]:
np.any(np.isnan(medians["dem"]))

False

In [20]:
medians["ndvi"]=np.true_divide(medians["nir"]-medians["red"],medians["nir"]+medians["red"])

In [21]:
medians["gndvi"]=np.true_divide(medians["nir"]-medians["green"],medians["nir"]+medians["green"])

In [22]:
medians["rvi"]=np.true_divide(medians["nir"],medians["red"])

In [23]:
#medians["nbr"]=np.true_divide(medians["nir"]-medians["swir1"],medians["nir"]+medians["swir1"])

In [24]:
#medians["nbr2"]=np.true_divide(medians["swir1"]-medians["swir2"],medians["swir1"]+medians["swir2"])

In [25]:
#medians["ndmi"]=np.true_divide(medians["nir"]-medians["swir1"],medians["nir"]+medians["swir1"])

In [26]:
bands=medians.keys()

In [27]:
_coords=xarr0.coords
_crs=xarr0.crs
del xarr0

In [28]:
files = [f for f in os.listdir(train_data_path) if f.endswith('.shp')]
classes = [f.split('.')[0] for f in files]
shapefiles = [os.path.join(train_data_path, f) for f in files if f.endswith('.shp')]

In [29]:
shapefiles

['/home/cubo/notebooks/975/Palma_975.shp',
 '/home/cubo/notebooks/975/No_Palma_975.shp']

In [30]:
rows, cols = medians[bands[0]].shape

In [31]:
#(originX, pixelWidth, 0, originY, 0, pixelHeight)
geo_transform=(_coords["longitude"].values[0], 0.000269995,0, _coords["latitude"].values[0],0,-0.000271302)

In [32]:
proj=_crs.wkt
#proj='GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

In [33]:
labeled_pixels = rasterizar_entrenamiento(shapefiles, rows, cols, geo_transform, proj)

In [34]:
labeled_pixels

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [35]:
is_train = np.nonzero(labeled_pixels)
training_labels = labeled_pixels[is_train]
nmed=None
bands_data=[]
for band in bands: 
    bands_data.append(medians[band])
bands_data = np.dstack(bands_data)
#training_samples = nmed[is_train]


In [36]:
rows, cols, n_bands = bands_data.shape

In [37]:
is_train = np.nonzero(labeled_pixels)
training_labels = labeled_pixels[is_train]
training_samples = bands_data[is_train]

In [38]:
_msk=np.sum(np.isfinite(training_samples),1)>1
training_samples= training_samples[_msk,:]
training_labels=training_labels[_msk]

In [39]:
training_samples.shape

(2672, 9)

In [40]:
classifier = RandomForestClassifier(n_jobs=-1, n_estimators=500, verbose=1, criterion="entropy" )
classifier.fit(training_samples, training_labels)

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.6s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [41]:
from sklearn.externals import joblib
joblib.dump(classifier,'modelo975')

['modelo975']

In [42]:
n_samples = rows*cols
flat_pixels = bands_data.reshape((n_samples, n_bands))

In [43]:
_msk=np.sum(np.isfinite(flat_pixels),1)>1
print _msk.shape

(13656648,)


In [44]:
result = classifier.predict(flat_pixels[_msk])
a=np.empty(rows*cols)
a[:]=np.nan
a[_msk]=result
classification = a.reshape((rows, cols))

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.7s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    3.7s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    8.8s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:   10.4s finished


In [45]:
#exportar("salida-class.tiff", classification, geo_transform, proj)

In [46]:
classification[np.isnan(classification)]=nodata

## Preparar la salida
La salida de los algoritmos puede expresarse como: 
- un xarray llamado output (que debe incluir entre sus atributos el crs del sistema de coordenadas)
- un diccionario con varios xarray llamado `outputs`
- Texto, en una variable llamada `outputtxt`

In [47]:
import xarray as xr
ncoords=[]
xdims =[]
xcords={}
for x in _coords:
    if(x!='time'):
        ncoords.append( ( x, _coords[x]) )
        xdims.append(x)
        xcords[x]=_coords[x]
        
variables={}
#variables ={k: xr.DataArray(v, dims=xdims,coords=ncoords)
#             for k, v in medians.items()}
variables["classification"]=xr.DataArray(classification,dims=xdims,coords=ncoords)
output=xr.Dataset(variables, attrs={'crs':_crs})

for x in output.coords:
    _coords[x].attrs["units"]=_coords[x].units

# Guardar la salida
La tarea genérica se encarga de generar los archivos de salida en la carpeta adecuada. 

__Nota__: A diferencia de la tarea genérica, que maneja los 3 tipos de salida descritos en la sección anterior, este cuaderno sólo guarda la salida definida en output

In [48]:
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
print "{}_{}_{}.nc".format(algorithm,version,execID)
nco=netcdf_writer.create_netcdf("{}_{}_{}.nc".format(algorithm,version,execID))
cords=('latitude', 'longitude','time')
for x in cords:
    if(x!="time"):
        netcdf_writer.create_coordinate(nco, x, _coords[x].values, _coords[x].units)
netcdf_writer.create_grid_mapping_variable(nco, _crs)
var= netcdf_writer.create_variable(nco, "classification", Variable(np.dtype(np.int32), nodata, ('latitude', 'longitude'), None) ,set_crs=True)
var[:] = netcdf_writer.netcdfy_data(classification)
nco.close()

RandomForestCm_1.0_975.nc
